In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [71]:
import pandas as pd
from html.parser import HTMLParser
import random
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

if 1:
    nltk.download('punkt')
    nltk.download('stopwords')
    !pip install stop-words

from stop_words import get_stop_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
myPath = 'drive/My Drive/geek_text/_1'

trainDf = pd.read_csv(myPath/train_tweets.csv')
testDf = pd.read_csv('myPath/test_tweets.csv')

In [4]:
trainDf.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation



1. Заменим html-сущности (к примеру: < > &). "<" заменим на “<” и "&" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

In [5]:
trainDf['crean_tweet'] = None
for i in range(trainDf.tweet.shape[0]):
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = HTMLParser().unescape(
        trainDf.loc[trainDf.index == i, 'tweet']
    )

# checking result
for i in random.sample(range(trainDf.shape[0]), 10):
    print(trainDf.loc[trainDf.index == i, 'tweet'])
    print(trainDf.loc[trainDf.index == i, 'clean_tweet'])
    print()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  after removing the cwd from sys.path.


2458    (the huffington post) a very   #father's day t...
Name: tweet, dtype: object
2458    (the huffington post) a very   #father's day t...
Name: clean_tweet, dtype: object

7759    so tonight, spagetti bolognaise and scrabble. ...
Name: tweet, dtype: object
7759    so tonight, spagetti bolognaise and scrabble. ...
Name: clean_tweet, dtype: object

19135    smile it's friday #weekend #friday #friends #e...
Name: tweet, dtype: object
19135    smile it's friday #weekend #friday #friends #e...
Name: clean_tweet, dtype: object

15008    lights on and now lights off.. thank you @user...
Name: tweet, dtype: object
15008    lights on and now lights off.. thank you @user...
Name: clean_tweet, dtype: object

10179    @user @user just looked at your timeline.  not...
Name: tweet, dtype: object
10179    @user @user just looked at your timeline.  not...
Name: clean_tweet, dtype: object

22490     @user new #storytime    third wheel?  #thirdw...
Name: tweet, dtype: object
22490     @user new #st

2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:
для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
для для замены @user на пробел, необходимо использовать re.sub() при применении функции необходимо использовать np.vectorize(function).


In [6]:
def cleanTweet(pattern, inputText):
    clean = re.compile(pattern)
    cleanTweet = re.sub(clean, '', inputText)

    return cleanTweet

pattern = '@[\w]*'

for i in range(trainDf.tweet.shape[0]):
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = cleanTweet(
        pattern,
        trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    )

# checking result
for i in random.sample(range(trainDf.shape[0]), 10):
    print(trainDf.loc[trainDf.index == i, 'tweet'])
    print(trainDf.loc[trainDf.index == i, 'clean_tweet'])
    print()

14297    @user we might be going to hell, but we're not...
Name: tweet, dtype: object
14297     we might be going to hell, but we're not surp...
Name: clean_tweet, dtype: object

20966    what's the final result?  #mindset   #joy #pro...
Name: tweet, dtype: object
20966    what's the final result?  #mindset   #joy #pro...
Name: clean_tweet, dtype: object

5311    its the hae tym u knw that life is sho no matt...
Name: tweet, dtype: object
5311    its the hae tym u knw that life is sho no matt...
Name: clean_tweet, dtype: object

18700    finally new car!!  #peugeot   
Name: tweet, dtype: object
18700    finally new car!!  #peugeot   
Name: clean_tweet, dtype: object

18663    my heas and prayers go out to all victims of o...
Name: tweet, dtype: object
18663    my heas and prayers go out to all victims of o...
Name: clean_tweet, dtype: object

23147    tonight's the night!  headed to see @user @use...
Name: tweet, dtype: object
23147    tonight's the night!  headed to see   in ptbo,...


3. Изменим регистр твитов на нижний с помощью .lower().

In [7]:
for i in range(trainDf.tweet.shape[0]):
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0].lower()

# checking result
for i in random.sample(range(trainDf.shape[0]), 10):
    print(trainDf.loc[trainDf.index == i, 'clean_tweet'])

14237    no matter how you pick it.   #flagday #flagday...
Name: clean_tweet, dtype: object
4526    i may finally get to go to my first  event thi...
Name: clean_tweet, dtype: object
30064    i #love diablo so much. he's my #boo.. #dog #r...
Name: clean_tweet, dtype: object
25895    i have begun something #new on this page &amp;...
Name: clean_tweet, dtype: object
19150    car wash with not so happy endingð¢ #carwash...
Name: clean_tweet, dtype: object
14507      niggas would fck over they boy just to hit s...
Name: clean_tweet, dtype: object
15020     â #oil under pressure, eyeing to test $47.0...
Name: clean_tweet, dtype: object
7490      #annoyed   #annoyed. all day everyday. 
Name: clean_tweet, dtype: object
28181    - - #never become   when #life pulls u back ð...
Name: clean_tweet, dtype: object
283    never been more appropriate: i have zero idea ...
Name: clean_tweet, dtype: object


4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [8]:
apostrophe_dict = {
    "ain't": "am not / are not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is",
    "i'd": "I had / I would",
    "i'd've": "I would have",
    "i'll": "I shall / I will",
    "i'll've": "I shall have / I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}

In [9]:
for i in range(trainDf.tweet.shape[0]):
    aTweet = trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = ' '.join(
        [apostrophe_dict.get(j, j) for j in aTweet.split()]
    )

5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [10]:
short_word_dict = {
    "121": "one to one",
    "a/s/l": "age, sex, location",
    "adn": "any day now",
    "afaik": "as far as I know",
    "afk": "away from keyboard",
    "aight": "alright",
    "alol": "actually laughing out loud",
    "b4": "before",
    "b4n": "bye for now",
    "bak": "back at the keyboard",
    "bf": "boyfriend",
    "bff": "best friends forever",
    "bfn": "bye for now",
    "bg": "big grin",
    "bta": "but then again",
    "btw": "by the way",
    "cid": "crying in disgrace",
    "cnp": "continued in my next post",
    "cp": "chat post",
    "cu": "see you",
    "cul": "see you later",
    "cul8r": "see you later",
    "cya": "bye",
    "cyo": "see you online",
    "dbau": "doing business as usual",
    "fud": "fear, uncertainty, and doubt",
    "fwiw": "for what it's worth",
    "fyi": "for your information",
    "g": "grin",
    "g2g": "got to go",
    "ga": "go ahead",
    "gal": "get a life",
    "gf": "girlfriend",
    "gfn": "gone for now",
    "gmbo": "giggling my butt off",
    "gmta": "great minds think alike",
    "h8": "hate",
    "hagn": "have a good night",
    "hdop": "help delete online predators",
    "hhis": "hanging head in shame",
    "iac": "in any case",
    "ianal": "I am not a lawyer",
    "ic": "I see",
    "idk": "I don't know",
    "imao": "in my arrogant opinion",
    "imnsho": "in my not so humble opinion",
    "imo": "in my opinion",
    "iow": "in other words",
    "ipn": "I’m posting naked",
    "irl": "in real life",
    "jk": "just kidding",
    "l8r": "later",
    "ld": "later, dude",
    "ldr": "long distance relationship",
    "llta": "lots and lots of thunderous applause",
    "lmao": "laugh my ass off",
    "lmirl": "let's meet in real life",
    "lol": "laugh out loud",
    "ltr": "longterm relationship",
    "lulab": "love you like a brother",
    "lulas": "love you like a sister",
    "luv": "love",
    "m/f": "male or female",
    "m8": "mate",
    "milf": "mother I would like to fuck",
    "oll": "online love",
    "omg": "oh my god",
    "otoh": "on the other hand",
    "pir": "parent in room",
    "ppl": "people",
    "r": "are",
    "rofl": "roll on the floor laughing",
    "rpg": "role playing games",
    "ru": "are you",
    "shid": "slaps head in disgust",
    "somy": "sick of me yet",
    "sot": "short of time",
    "thanx": "thanks",
    "thx": "thanks",
    "ttyl": "talk to you later",
    "u": "you",
    "ur": "you are",
    "uw": "you’re welcome",
    "wb": "welcome back",
    "wfm": "works for me",
    "wibni": "wouldn't it be nice if",
    "wtf": "what the fuck",
    "wtg": "way to go",
    "wtgp": "want to go private",
    "ym": "young man",
    "gr8": "great"
}

In [11]:
for i in range(trainDf.tweet.shape[0]):
    aTweet = trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = ' '.join(
        [short_word_dict.get(j, j) for j in aTweet.split()]
    )

6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [12]:
emoticon_dict = {
    ":)": "happy",
    ":‑)": "happy",
    ":-]": "happy",
    ":-3": "happy",
    ":->": "happy",
    "8-)": "happy",
    ":-}": "happy",
    ":o)": "happy",
    ":c)": "happy",
    ":^)": "happy",
    "=]": "happy",
    "=)": "happy",
    "<3": "happy",
    ":-(": "sad",
    ":(": "sad",
    ":c": "sad",
    ":<": "sad",
    ":[": "sad",
    ">:[": "sad",
    ":{": "sad",
    ">:(": "sad",
    ":-c": "sad",
    ":-< ": "sad",
    ":-[": "sad",
    ":-||": "sad"
}

In [13]:
for i in range(trainDf.tweet.shape[0]):
    aTweet = trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = ' '.join(
        [emoticon_dict.get(j, j) for j in aTweet.split()]
    )

7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

In [14]:
def cleanTweet(pattern, inputText):
    clean = re.compile(pattern)
    cleanTweet = re.sub(clean, ' ', inputText)

    return cleanTweet

pattern = '[^\w\s]'

for i in range(trainDf.tweet.shape[0]):
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = cleanTweet(
        pattern,
        trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    )


8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [15]:
def cleanTweet(pattern, inputText):
    clean = re.compile(pattern)
    cleanTweet = re.sub(clean, ' ', inputText)

    return cleanTweet

pattern = '[^a-zA-Z0-9]'

for i in range(trainDf.tweet.shape[0]):
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = cleanTweet(
        pattern,
        trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    )


9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

In [16]:
def cleanTweet(pattern, inputText):
    clean = re.compile(pattern)
    cleanTweet = re.sub(clean, ' ', inputText)

    return cleanTweet

pattern = '[^a-zA-Z]'

for i in range(trainDf.tweet.shape[0]):
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = cleanTweet(
        pattern,
        trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    )

10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).


In [17]:
for i in range(trainDf.tweet.shape[0]):
    aTweet = trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    trainDf.loc[trainDf.index == i, 'clean_tweet'] = ' '.join(
        [w for w in aTweet.split() if len(w) > 1]
    )

11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [33]:
trainDf['tweet_token'] = 0

for i in range(trainDf.tweet.shape[0]):
    aTweet = trainDf.loc[trainDf.index == i, 'clean_tweet'].values[0]
    trainDf.tweet_token[i] = word_tokenize(aTweet)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0        [when, father, is, dysfunctional, and, is, so,...
1        [thanks, for, lyft, credit, can, not, use, cau...
2                                  [bihday, your, majesty]
3        [model, love, you, take, with, you, all, the, ...
4                   [factsguide, society, now, motivation]
                               ...                        
31957                              [ate, isz, that, youuu]
31958    [to, see, nina, turner, on, the, airwaves, try...
31959    [listening, to, sad, songs, on, monday, mornin...
31960    [sikh, temple, vandalised, in, in, calgary, ws...
31961                       [thank, you, for, you, follow]
Name: tweet_token, Length: 31962, dtype: object

12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [63]:
stopWords = set(stopwords.words('english'))

trainDf['tweet_token_filtered'] = 0

for i in range(trainDf.tweet.shape[0]):
    aToken = trainDf.loc[trainDf.index == i, 'tweet_token'].values[0]
    trainDf.tweet_token_filtered[i] = [w for w in aToken if not w in stopWords]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [96]:
stemmer = PorterStemmer()
trainDf['tweet_stemmed'] = 0
for i in range(trainDf.shape[0]):
    trainDf.tweet_stemmed[i] = [stemmer.stem(w) for w in trainDf.tweet_token_filtered[i]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [98]:
lemmatizer = WordNetLemmatizer()
trainDf['tweet_lemmatized'] = 0
for i in range(trainDf.shape[0]):
    trainDf.tweet_lemmatized[i] = [stemmer.stem(w) for w in trainDf.tweet_token_filtered[i]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


15. Сохраним результат предобработки в pickle-файл.

In [103]:
trainDf.to_pickle(myPath+'trainProc.pkl')